In [6]:
#Librerias

import pandas as pd
import numpy as np

# **Entrega 1**

El mapa de Medellín es un archivo donde las columnas
son separadas por puntos y comas (CSV). Almacenar el mapa en una estructura de datos; por ejemplo, una matriz (de Numpy), una lista de Python, un diccionario (dataframe de Pandas) u otra.

In [7]:
data = pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/calles_de_medellin_con_acoso.csv", sep=";")
data

,name,origin,destination,length,oneway,harassmentRisk,geometry
0,Calle 10,"(-75.5728593, 6.2115169)","(-75.5724985, 6.2113756)",42.867,True,0.526539,"LINESTRING (-75.5728593 6.2115169, -75.5724984..."
1,Calle 10,"(-75.5705202, 6.2106275)","(-75.570427, 6.2105879)",11.204,True,0.414356,"LINESTRING (-75.5705202 6.2106275, -75.570427 ..."
2,Carrera 43A,"(-75.5705202, 6.2106275)","(-75.5705604, 6.2105262)",12.109,True,0.526539,"LINESTRING (-75.5705202 6.2106275, -75.5705604..."
3,Carrera 41,"(-75.5687719, 6.2099661)","(-75.5688022, 6.2098867)",9.443,True,0.302173,"LINESTRING (-75.5687719 6.2099661, -75.5688021..."
4,Calle 10,"(-75.5687719, 6.2099661)","(-75.568715, 6.2099443)",6.741,True,0.302173,"LINESTRING (-75.5687719 6.2099661, -75.568715 ..."
...,...,...,...,...,...,...,...
68744,Medellín - San Jerónimo,"(-75.7086893, 6.3627057)","(-75.7088895, 6.3608552)",207.573,False,NaN,"LINESTRING (-75.7086893 6.3627057, -75.7087245..."
68745,NaN,"(-75.7086893, 6.3627057)","(-75.708837, 6.3625464)",24.181,False,NaN,"LINESTRING (-75.7086893 6.3627057, -75.7087739..."
68746,San Jerónimo-Medellín,"(-75.6909483, 6.338773)","(-75.6898219, 6.33719)",270.387,False,NaN,"LINESTRING (-75.6909483 6.338773, -75.6902973 ..."
68747,"['Medellín - San Jerónimo', 'San Jerónimo-Mede...","(-75.6909483, 6.338773)","(-75.7086472, 6.3610709)",3855.673,False,NaN,"LINESTRING (-75.6909483 6.338773, -75.6918575 ..."


# **Entrega 2**

Incluir la implementación de un algoritmo, seleccionado
de las alternativas de solución, para resolver el primer problema.def

In [ ]:
class Graph:
    
    def __init__(self, data, metric='length'): #OJO: En esta linea se indica la variable para la que quiero hallar el camino más corto: Distancia o Acoso
        self.data = data
        self.metric = metric
        self.generate_nodes()
        self.generate_graph()
        
    def generate_nodes(self):
        if self.data is None:
            raise Exception("There's no data")

        origin_nodes = data['origin'].to_numpy()
        destination_nodes = data['destination'].to_numpy()
        all_nodes = np.concatenate([origin_nodes, destination_nodes])
        unique_nodes = np.unique(all_nodes)
        nodes = pd.DataFrame({'coordinates': unique_nodes})
        nodes['id'] = nodes.index
        nodes['id'] = nodes['id'].apply(str)
        self.nodes = nodes.set_index('coordinates')
        
    def generate_graph(self):
        self.graph = { node: {} for node in self.nodes['id'] }
        for index, row in self.data.iterrows():
            origin = self.nodes.loc[str(row['origin'])]['id']
            dest = self.nodes.loc[str(row['destination'])]['id']
            metric = row[self.metric]
            self.graph[origin][dest] = metric
    
    def calculate_distances_from(self, start):
        if start not in self.graph.keys():
            raise Exception(f"Node '{start}' does not exist")
        path = {}
        self.adj_node = {}
        queue = []
        for node in self.graph:
            path[node] = float("inf")
            self.adj_node[node] = None
            queue.append(node)
        path[start] = 0
        while queue:
            key_min = queue[0]
            min_val = path[key_min]
            for n in range(1, len(queue)):
                if path[queue[n]] < min_val:
                    key_min = queue[n]
                    min_val = path[key_min]
            cur = key_min
            queue.remove(cur)

            for i in self.graph[cur]:
                alternate = self.graph[cur][i] + path[cur]
                if path[i] > alternate:
                    path[i] = alternate
                    self.adj_node[i] = cur

    def get_path_to(self, node, show_coordinates=False):
        if node not in self.graph.keys():
            raise Exception(f"Node '{node}' does not exist")
            
        if not self.adj_node:
            raise Exception("Distances have not been calculated")
        
        if show_coordinates:
            nodes = self.nodes.copy()
            nodes['coords'] = nodes.index
            nodes = nodes.set_index('id')
            
        path = [nodes.loc[node][0]] if show_coordinates else [node]
        while True:
            node = self.adj_node[node]
            if node is None:
                path.reverse()
                break
            path.append(nodes.loc[node][0] if show_coordinates else node)
        return path

In [ ]:
graph = Graph(data)
graph.calculate_distances_from('2')

In [ ]:
path = "→".join(graph.get_path_to("1000", show_coordinates=False))
print('The shortest path is:')
print(path)

The shortest path is:
2→1→3→4→7→13→19→25→26→30→32→50→53→63→69→66→67→107→149→248→334→363→388→401→413→432→438→444→483→497→499→516→546→548→576→585→610→648→914→1052→1090→1066→1062→1035→1013→1000


# **Entrega 3**

Solución final

In [8]:
"""
{
  "A": {"B": {'harassment': 0.5, 'distance': 10, ...}}

}


"""

class Graph:
    def __init__(self):
        self.graph = {}
    
    def add(self, node1, node2, value):
        self.graph.setdefault(node1, {}).setdefault(node2, value)
        self.graph.setdefault(node2, {}).setdefault(node1, value)

    def get_nodes(self):
        return self.graph.keys()

    def show_path(self, source, destination, paths):
        current = destination
        path = [current]
        distance = 0
        harassment = 0
        n = 0
        while current != source:
            costs = self.graph[current][paths[current]]
            distance += costs['distance']
            if not np.isnan(harassment):
                harassment += costs['harassment']
                n += 1
            current = paths[current]
            path.append(current)
        path.reverse()
        return path, distance, harassment/n

    def get_path_from(self, source, metric='distance'):
        max_val = float("Inf")
        unvisited = list(self.get_nodes())
        path = {}
        costs = { node: max_val for node in unvisited }
        costs[source] = 0
        while unvisited:
            current_min = None
            for node in unvisited:
                if current_min == None or costs[node] < costs[current_min]:
                    current_min = node
            
            neighbors = self.graph[current_min]
            for neighbor,cost in neighbors.items():
                tentative_value = costs[current_min] + cost[metric]
                if tentative_value < costs[neighbor]:
                    costs[neighbor] = tentative_value
                    path[neighbor] = current_min
            unvisited.remove(current_min)
        return costs, path


In [9]:
graph = Graph()
harassment_weight = 0.85
distance_weight = 5_000
harassment_mean = data['harassmentRisk'].mean(skipna=True)
metric = 'distance+harassment'

for index, row in data.iterrows():
    harassment = row['harassmentRisk']
    harassment = harassment_mean if np.isnan(harassment) else harassment
    distance = row['length']
    distance_harassment = harassment*harassment_weight + distance
    harassment_distance = distance*distance_weight + harassment
    graph.add(row['origin'], row['destination'], {
        'distance': distance,
        'harassment': harassment,
        'distance+harassment': distance_harassment,
        'harassment+distance': harassment_distance
    })

In [10]:
origin = '(-75.5778046, 6.2029412)'

costs, paths = graph.get_path_from(origin, metric=metric)

In [11]:
dest = '(-75.6101004, 6.2312125)'
path, distance, harassment = graph.show_path(origin, dest, paths)


print(' → '.join(path))
print()
if metric in ['harassment', 'harassment+distance']:
  print(distance)
elif metric in ['distance', 'distance+harassment']:
  print(harassment)

(-75.5778046, 6.2029412) → (-75.5778053, 6.2027845) → (-75.5777932, 6.2026166) → (-75.5778113, 6.2024225) → (-75.5778475, 6.2022372) → (-75.5778756, 6.2019281) → (-75.577875, 6.2018479) → (-75.5774434, 6.2013463) → (-75.5773228, 6.2024694) → (-75.577303, 6.2026793) → (-75.5772946, 6.2027679) → (-75.5773822, 6.2028635) → (-75.5773507, 6.2033125) → (-75.5774238, 6.2033518) → (-75.5775041, 6.2033481) → (-75.5800756, 6.2044069) → (-75.5800271, 6.20488) → (-75.5799948, 6.2049265) → (-75.5802571, 6.2043718) → (-75.5809586, 6.204664) → (-75.5814131, 6.2048544) → (-75.5820081, 6.2050967) → (-75.5820673, 6.2051209) → (-75.5821445, 6.2051523) → (-75.5827708, 6.2054074) → (-75.5834388, 6.2056794) → (-75.5840019, 6.2059088) → (-75.5840651, 6.2059345) → (-75.584126, 6.2059593) → (-75.5847428, 6.2062105) → (-75.5854194, 6.2064861) → (-75.5859755, 6.2067126) → (-75.5860605, 6.2067374) → (-75.5861627, 6.2067672) → (-75.5862441, 6.206791) → (-75.5867083, 6.2069872) → (-75.5878053, 6.207451) → (-75.5884